In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import math
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from CustomDataset import CustomDataset
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score
import tqdm

torch.manual_seed(0)

In [2]:
# Create the dataloader
normalize = transforms.Normalize(mean=[0.485,0.456,0.406],
                    std=[0.229,0.224,0.225])
data_transforms = {
    'Train':
    transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]),
    'Validation':
    transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]),
}
dataset = CustomDataset(target_type = 'poly')

batch_size = 2
TRAIN_SIZE= math.floor(dataset.__len__()*0.70)
VALIDATION_SIZE = math.floor((dataset.__len__() - TRAIN_SIZE)/2)
TEST_SIZE = math.floor(dataset.__len__() - (TRAIN_SIZE+VALIDATION_SIZE))
print([dataset.__len__(),TRAIN_SIZE, VALIDATION_SIZE, TEST_SIZE])
image_datasets = {
    'Train': 
        TRAIN_SIZE,
    'Validation':
        VALIDATION_SIZE,
    'Test':
        TEST_SIZE}

trainset, validationset, testset = random_split(dataset,[TRAIN_SIZE, VALIDATION_SIZE, TEST_SIZE])
test_dataloader = DataLoader(testset, batch_size=1, shuffle=True)



[1155, 808, 173, 174]


In [3]:
test_dataloader.__len__()

174

In [4]:
# Evaluate
model = torch.load('models/pytorch/weights_deeplab_lr0.0001_beta_0.7_0.9999_200epochs.pt')
model.eval();
model.to("cpu");

In [ ]:
def evaluate():
    running_loss = 0
    metrics_roc = 0
    model.eval();
    for (t_image, mask) in tqdm.tqdm(iter(test_dataloader)):
        outputs = model(t_image)["out"] # forward
        
        preds = torch.squeeze(outputs).float()
        
        running_loss += torchvision.ops.sigmoid_focal_loss(preds, torch.squeeze(mask), reduction = 'mean')
        #print(running_loss)
        try:
            metrics_roc += roc_auc_score(mask.int().ravel().detach().numpy(),preds.ravel().numpy())
        except:
            metrics_roc += 0.5
    return running_loss/test_dataloader.__len__(), metrics_roc
loss, roc = evaluate()

 30%|███       | 53/174 [00:55<02:43,  1.35s/it]

In [ ]:
def plot_images_eval(n, DataLoader = test_dataloader):
    for batch, (t_image, mask) in enumerate(DataLoader):
        outputs = model(t_image)["out"] # forward
        _, predicted = torch.max(outputs.data, 1)
        plt1 = plt.figure()
        plt1 = plt.imshow(predicted[0])
        plt2 = plt.figure()
        plt2 = plt.imshow(mask.data[0])
        print(batch)
        if batch+1 == n:
            break
plot_images_eval(1,test_dataloader)

In [ ]:
# hyper parameter evaluation:
def plot_loss_hpt(path_to_dir):
    filenames = os.listdir(path_to_dir)
    filenames = [ os.path.join(path_to_dir, filename) for filename in filenames if filename.endswith(".csv") ]
    y_low = 1
    for filename in filenames:
        y = pd.read_csv(filename)["Validation_loss"].values
        #print(filename, y)
        x = np.arange(0,len(y))+1
        name = os.path.splitext(os.path.basename(filename))[0]
        #print(name, y)
        #with plt.style.context("fivethirtyeight"):
        line = plt.plot(x,y,label=name)
        if y[-1] < y_low:
            y_low = y[-1]
            filename_low = filename
            
    print(filename_low, y_low)
    plt.legend()
    #plt.rcParams['figure.figsize'] = [40, 2]
    plt.show()
    plt.savefig("hpt_plot.png")
exp_directory = 'models/pytorch/hpt/'
plot_loss_hpt(exp_directory)